# Data Quality Starter

### Case 3: Classify email domains
You need to clasify the email addresses that have been entered on our website to determine if they a work email address or a personal email address.

In [ ]:
select email,
 snowflake.cortex.complete('mistral-large'
        , [
        {'role': 'system', 'content':
            'You are a marketing expert and your job is to classify the email addresses that have been entered on your website to determine if they are a likely to be a work email address or a personal email address, or to flag that you are unsure if it should be examined more closely.'
            || 'Email addresses have two parts - a username and a domain. Both parts should be considered to evaluate the email addresses. '
            || 'Business emails likely have either a username or a domain that indicates a business. It could be a large, recognizable business, or a more descriptive small business name. A username that looks like a small business should be considered a business email even on a free email domain.'
            || 'Personal emails have usernames that are typically based on the name or nickname of a person, often with numbers. Domains are often internet service providers or free email services like gmail, yahoo, etc., or custom domains, with variations of names or families, without the mention of a business.'
            || 'Note that some companies like Comcast and AT&T would have both business emails and personal emails, with employees using the "dot com" domain (comcast.com, att.com) and customers using the "dot net" domain (comcast.net, att.net). Additionally, some companies like Google and Microsoft use different domains for their corporate employees (dot com domain) than for their free email services (Gmail and Hotmail).'
            || 'Wrap the category classification with an xml tag <response>.'
            || 'You should provide only "business" or "personal" and no text other than the xml tag' },
        {'role': 'user', 'content': '<data>'
            || ' username: '
            || LEFT(email, CHARINDEX('@', email) - 1)
            || ' , domain: '
            || RIGHT(email, LEN(email) - CHARINDEX('@', email))
            || '</data>'}
            ]
        , {'max_tokens': 20}
        ):choices[0]:messages::string as classification_raw
    , regexp_substr(classification_raw, '<response>(.*?)<\/response>', 1, 1, 'e', 1) as classification
from LLM_DEMO.DEMO.customer_emails;